<a href="https://colab.research.google.com/github/srikanthgr/pyspark-python/blob/main/pyspark_udf_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.6.tgz
!tar -xvf spark-2.4.3-bin-hadoop2.6.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.6"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [2]:
df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()

/content/spark-2.4.3-bin-hadoop2.6/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



In [3]:
!wget https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_customers.csv

--2021-03-05 06:30:32--  https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_customers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 199132 (194K) [text/plain]
Saving to: ‘store_customers.csv’

store_customers.csv 100%[===================>] 194.46K  --.-KB/s    in 0.03s   

2021-03-05 06:30:33 (6.42 MB/s) - ‘store_customers.csv’ saved [199132/199132]



In [4]:
!wget https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_transactions.csv

--2021-03-05 06:31:12--  https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_transactions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7257374 (6.9M) [text/plain]
Saving to: ‘store_transactions.csv’

store_transactions. 100%[===================>]   6.92M  39.0MB/s    in 0.2s    

2021-03-05 06:31:13 (39.0 MB/s) - ‘store_transactions.csv’ saved [7257374/7257374]



In [5]:
!ls 

sample_data/                   store_customers.csv
spark-2.4.3-bin-hadoop2.6/     store_transactions.csv
spark-2.4.3-bin-hadoop2.6.tgz


In [7]:
customerDf = spark.read.csv("store_customers.csv", header=True)
customerDf.show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         3| 70| 24000|Female|England|
|         4| 75|  2600|  Male|England|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
|         9| 76| 35000|  Male|Germany|
|        10| 58| 37000|Female| France|
|        11| 70| 25000|  Male|Germany|
|        12| 28| 27000|Female| France|
|        13| 21| 29000|Female|England|
|        14| 34|  7600|  Male|England|
|        15| 45| 55000|  Male| France|
|        16| 32| 40000|Female|England|
|        17| 62|  9300|  Male|Germany|
|        18| 54| 37000|Female| France|
|        19| 33| 40000|  Male|Germany|
|        20| 46| 42000|Female| France|
+----------+---+------+------+-------+
only showing top 20 rows



In [8]:
customerDf.count()

7000

In [9]:
transactionDF = spark.read.csv("store_transactions.csv", header=True)
transactionDF.show()

+----------+---------+------+----------+
|CustomerID|ProductID|Amount|      Date|
+----------+---------+------+----------+
|      3427|        3|  7541|22-11-2019|
|      4378|       14|  7271|15-12-2019|
|      3751|       47|  4276|20-11-2019|
|      6899|      146|  8923|22-11-2019|
|      4561|       46|  4891|30-11-2019|
|      2299|      143|  7545|05-12-2019|
|       553|       43|  1147|08-12-2019|
|      3406|      134|   245|12-12-2019|
|      5278|       34|  8765|09-12-2019|
|      2456|       68|  3820|30-11-2019|
|      6963|       58|  1850|27-11-2019|
|      4700|      140|  3948|27-11-2019|
|      3566|       76|   401|17-12-2019|
|      2212|       38|  4750|16-12-2019|
|      4677|       18|  1149|18-11-2019|
|      4295|       35|  3241|19-11-2019|
|      4228|       91|  6405|01-12-2019|
|      2466|       83|  2978|10-12-2019|
|      3136|       67|  6581|28-11-2019|
|      6563|       80|  2878|21-11-2019|
+----------+---------+------+----------+
only showing top

In [10]:
transactionDF.count()

1048575

In [13]:
from pyspark.sql.functions import udf

extract_year = udf (lambda Date:Date.split('-')[2])

transactionDF = transactionDF.withColumn("year",extract_year(transactionDF.Date))

transactionDF.show()

+----------+---------+------+----------+----+
|CustomerID|ProductID|Amount|      Date|year|
+----------+---------+------+----------+----+
|      3427|        3|  7541|22-11-2019|2019|
|      4378|       14|  7271|15-12-2019|2019|
|      3751|       47|  4276|20-11-2019|2019|
|      6899|      146|  8923|22-11-2019|2019|
|      4561|       46|  4891|30-11-2019|2019|
|      2299|      143|  7545|05-12-2019|2019|
|       553|       43|  1147|08-12-2019|2019|
|      3406|      134|   245|12-12-2019|2019|
|      5278|       34|  8765|09-12-2019|2019|
|      2456|       68|  3820|30-11-2019|2019|
|      6963|       58|  1850|27-11-2019|2019|
|      4700|      140|  3948|27-11-2019|2019|
|      3566|       76|   401|17-12-2019|2019|
|      2212|       38|  4750|16-12-2019|2019|
|      4677|       18|  1149|18-11-2019|2019|
|      4295|       35|  3241|19-11-2019|2019|
|      4228|       91|  6405|01-12-2019|2019|
|      2466|       83|  2978|10-12-2019|2019|
|      3136|       67|  6581|28-11

In [15]:
country_spend_details = customerDf.join(transactionDF, customerDf.CustomerID == transactionDF.CustomerID)
country_spend_details.show()

+----------+---+------+------+-------+----------+---------+------+----------+----+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|year|
+----------+---+------+------+-------+----------+---------+------+----------+----+
|      3427| 69| 14300|Female|England|      3427|        3|  7541|22-11-2019|2019|
|      4378| 78| 42000|  Male|Germany|      4378|       14|  7271|15-12-2019|2019|
|      3751| 57| 45000|Female|England|      3751|       47|  4276|20-11-2019|2019|
|      6899| 34| 12600|  Male|Germany|      6899|      146|  8923|22-11-2019|2019|
|      4561| 37| 50000|Female|Germany|      4561|       46|  4891|30-11-2019|2019|
|      2299| 50| 12600|Female| France|      2299|      143|  7545|05-12-2019|2019|
|       553| 29| 42000|  Male|Germany|       553|       43|  1147|08-12-2019|2019|
|      3406| 37| 35000|Female|England|      3406|      134|   245|12-12-2019|2019|
|      5278| 64| 42000|Female|England|      5278|       34|  8765|09-12-2019|2019|
|   

In [16]:
country_spend_details.groupBy("Country").agg({"Amount":"sum"}).show()

+-------+------------+
|Country| sum(Amount)|
+-------+------------+
|Germany|1.34377551E8|
| France|1.45868556E8|
|England|2.30855023E8|
+-------+------------+



In [17]:
!wget https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_customers_mini.csv

--2021-03-05 06:42:00--  https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_customers_mini.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [text/plain]
Saving to: ‘store_customers_mini.csv’

store_customers_min 100%[===================>]     190  --.-KB/s    in 0s      

2021-03-05 06:42:00 (12.5 MB/s) - ‘store_customers_mini.csv’ saved [190/190]



In [18]:
!wget https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_transactions_mini.csv

--2021-03-05 06:42:16--  https://raw.githubusercontent.com/srikanthgr/pyspark-python/main/store_transactions_mini.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167 [text/plain]
Saving to: ‘store_transactions_mini.csv’

store_transactions_ 100%[===================>]     167  --.-KB/s    in 0s      

2021-03-05 06:42:16 (6.43 MB/s) - ‘store_transactions_mini.csv’ saved [167/167]



In [19]:
!ls

sample_data		       store_customers_mini.csv
spark-2.4.3-bin-hadoop2.6      store_transactions.csv
spark-2.4.3-bin-hadoop2.6.tgz  store_transactions_mini.csv
store_customers.csv


In [22]:
customerDFMini = spark.read.csv("store_customers_mini.csv", header=True)
customerDFMini.show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
+----------+---+------+------+-------+



In [23]:
transactionDFMini = spark.read.csv("store_transactions_mini.csv",header=True)
transactionDFMini.show()

+----------+---------+------+----------+
|CustomerID|ProductID|Amount|      Date|
+----------+---------+------+----------+
|         1|        3|  7541|22-11-2019|
|         2|       14|  7271|15-12-2019|
|         3|       47|  4276|20-11-2019|
|         4|      146|  8923|22-11-2019|
|         5|       46|  4891|30-11-2019|
|         6|      143|  7545|05-12-2019|
+----------+---------+------+----------+



In [25]:
customerDFMini.join(transactionDFMini, customerDFMini.CustomerID == transactionDFMini.CustomerID).show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+---+------+------+-------+----------+---------+------+----------+



In [26]:
customerDFMini.join(transactionDFMini, customerDFMini.CustomerID == transactionDFMini.CustomerID, how= "inner").show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+---+------+------+-------+----------+---------+------+----------+



In [27]:
customerDFMini.join(transactionDFMini, customerDFMini.CustomerID == transactionDFMini.CustomerID, how="left").show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2| 72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6| 52| 35000|Female|England|         6|      143|  7545|05-12-2019|
|         7| 31|  4300|  Male|Germany|      null|     null|  null|      null|
|         8| 37| 32000|Female| France|      null|     null|  null|      null|
+----------+---+------+------+-------+----------+---------+------+----------+



In [28]:
customerDFMini.join(transactionDFMini, customerDFMini.CustomerID == transactionDFMini.CustomerID, how="right").show()

+----------+----+------+------+-------+----------+---------+------+----------+
|CustomerID| Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+----+------+------+-------+----------+---------+------+----------+
|         1|  72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|         2|  72| 22000|Female| France|         2|       14|  7271|15-12-2019|
|      null|null|  null|  null|   null|         3|       47|  4276|20-11-2019|
|      null|null|  null|  null|   null|         4|      146|  8923|22-11-2019|
|         5|  33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6|  52| 35000|Female|England|         6|      143|  7545|05-12-2019|
+----------+----+------+------+-------+----------+---------+------+----------+



In [29]:
customerDFMini.join(transactionDFMini, customerDFMini.CustomerID == transactionDFMini.CustomerID, how="full").show()

+----------+----+------+------+-------+----------+---------+------+----------+
|CustomerID| Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+----+------+------+-------+----------+---------+------+----------+
|         7|  31|  4300|  Male|Germany|      null|     null|  null|      null|
|      null|null|  null|  null|   null|         3|       47|  4276|20-11-2019|
|         8|  37| 32000|Female| France|      null|     null|  null|      null|
|         5|  33| 50000|  Male| France|         5|       46|  4891|30-11-2019|
|         6|  52| 35000|Female|England|         6|      143|  7545|05-12-2019|
|         1|  72| 20000|  Male|Germany|         1|        3|  7541|22-11-2019|
|      null|null|  null|  null|   null|         4|      146|  8923|22-11-2019|
|         2|  72| 22000|Female| France|         2|       14|  7271|15-12-2019|
+----------+----+------+------+-------+----------+---------+------+----------+



In [30]:
customerDFMini.join(transactionDFMini,customerDFMini.CustomerID == transactionDFMini.CustomerID, how="left_semi" ).show()

+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         1| 72| 20000|  Male|Germany|
|         2| 72| 22000|Female| France|
|         5| 33| 50000|  Male| France|
|         6| 52| 35000|Female|England|
+----------+---+------+------+-------+



In [31]:
customerDFMini.join(transactionDFMini,customerDFMini.CustomerID == transactionDFMini.CustomerID, how="left_anti" ).show()


+----------+---+------+------+-------+
|CustomerID|Age|Salary|Gender|Country|
+----------+---+------+------+-------+
|         7| 31|  4300|  Male|Germany|
|         8| 37| 32000|Female| France|
+----------+---+------+------+-------+



In [32]:
customerDFMini.join(transactionDFMini,customerDFMini.CustomerID > transactionDFMini.CustomerID,how="inner" ).show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         2| 72| 22000|Female| France|         1|        3|  7541|22-11-2019|
|         5| 33| 50000|  Male| France|         1|        3|  7541|22-11-2019|
|         5| 33| 50000|  Male| France|         2|       14|  7271|15-12-2019|
|         5| 33| 50000|  Male| France|         3|       47|  4276|20-11-2019|
|         5| 33| 50000|  Male| France|         4|      146|  8923|22-11-2019|
|         6| 52| 35000|Female|England|         1|        3|  7541|22-11-2019|
|         6| 52| 35000|Female|England|         2|       14|  7271|15-12-2019|
|         6| 52| 35000|Female|England|         3|       47|  4276|20-11-2019|
|         6| 52| 35000|Female|England|         4|      146|  8923|22-11-2019|
|         6| 52| 35000|Female|England|         5|       46|  489

In [33]:
customerDFMini.join(transactionDFMini,customerDFMini.CustomerID < transactionDFMini.CustomerID,how="inner" ).show()

+----------+---+------+------+-------+----------+---------+------+----------+
|CustomerID|Age|Salary|Gender|Country|CustomerID|ProductID|Amount|      Date|
+----------+---+------+------+-------+----------+---------+------+----------+
|         1| 72| 20000|  Male|Germany|         2|       14|  7271|15-12-2019|
|         1| 72| 20000|  Male|Germany|         3|       47|  4276|20-11-2019|
|         1| 72| 20000|  Male|Germany|         4|      146|  8923|22-11-2019|
|         1| 72| 20000|  Male|Germany|         5|       46|  4891|30-11-2019|
|         1| 72| 20000|  Male|Germany|         6|      143|  7545|05-12-2019|
|         2| 72| 22000|Female| France|         3|       47|  4276|20-11-2019|
|         2| 72| 22000|Female| France|         4|      146|  8923|22-11-2019|
|         2| 72| 22000|Female| France|         5|       46|  4891|30-11-2019|
|         2| 72| 22000|Female| France|         6|      143|  7545|05-12-2019|
|         5| 33| 50000|  Male| France|         6|      143|  754